<a href="https://colab.research.google.com/github/MangoHaha/ImageRecognition/blob/master/ImageRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Linked to my google drive for output dir

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
'2017Fall Reimbursement'
'#7_HW3_FengGao.gdoc'
 AI
 Chenchen-1040-Form-2015.pdf
 Chenchen-1040-Form-2016.pdf
 Chenchen-401k-Statement-Oct.pdf
 Chenchen-401k-Statement-Sept.pdf
 Chenchen-BOA-eStmt_2017-11-16.pdf
 Chenchen-BOA-eStmt_2017-12-15.pdf
 Chenchen-Chase-2017-09-11-statements-9628.pdf
 Chenchen-Chase-2017-10-10-statements-9628.pdf
 Chenchen-Chase-2017-11-08-statements-9628.pdf
 Chenchen-Chase-2017-12-08-statements-9628.pdf
 Chenchen-CITI-11-16-2017.pdf


Import all necessary libraries

In [2]:
import numpy as np
import pandas as pd
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Bidirectional
from sklearn.model_selection import train_test_split
from keras.metrics import categorical_accuracy
from keras import backend as K
import tensorflow as tf
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Flatten
from sklearn.model_selection import KFold


Using TensorFlow backend.


Define helper methods

In [0]:

# The custom accuracy metric used for this task
def customAccuracy(y_true, y_pred):
    y = tf.argmax(y_true, axis =- 1)
    y_ = tf.argmax(y_pred, axis =- 1)
    mask = tf.greater(y, 0)
    return K.cast(K.equal(tf.boolean_mask(y, mask), tf.boolean_mask(y_, mask)), K.floatx())

# Maps the sequence to a one-hot encoding
def onehot_to_seq(oh_seq, index):
    s = ''
    for o in oh_seq:
        i = np.argmax(o)
        if i != 0:
            s += index[i]
        else:
            break
    return s

# prints the results
def print_results(x, y_, revsere_decoder_index):
    # print("input     : " + str(x))
    # print("prediction: " + str(onehot_to_seq(y_, revsere_decoder_index).upper()))
    print(str(x), str(onehot_to_seq(y_, revsere_decoder_index).upper()))

# Computes and returns the n-grams of a particualr sequence, defaults to trigrams
def seq2ngrams(seqs, n = 3):
    return np.array([[seq[i : i + n] for i in range(len(seq))] for seq in seqs])

# print output to a file in google drive
def write_to_file(test_df, y_test_pred):
  f = open('output.csv','w')
  f.write("id" + ',' + "expected")
  for i in range(len(test_df)):
    f.write('\n' + test_df['id'].iloc[i] +',' + str(onehot_to_seq(y_test_pred[i], revsere_decoder_index).upper()))
  f.close()


Red train and test csv files

In [8]:
train_df = pd.read_csv('/content/drive/My Drive/train.csv')
test_df = pd.read_csv('/content/drive/My Drive/test.csv')
mega = pd.ExcelFile('/content/drive/My Drive/megamilionshistoryOD.xls')

ImportError: ignored

Prepare data for training

In [0]:
maxlen_seq = 2048
# Loading and converting the inputs to trigrams
train_input_seqs, train_target_seqs = train_df[['input', 'expected']][(train_df.len <= maxlen_seq)].values.T
train_input_grams = seq2ngrams(train_input_seqs)

# Same for test
test_input_seqs = test_df['input'].values.T
test_input_grams = seq2ngrams(test_input_seqs)

# Initializing and defining the tokenizer encoders and decoders based on the train set
tokenizer_encoder = Tokenizer()
tokenizer_encoder.fit_on_texts(train_input_grams)
tokenizer_decoder = Tokenizer(char_level = True)
tokenizer_decoder.fit_on_texts(train_target_seqs)

# Using the tokenizer to encode and decode the sequences for use in training
# Inputs
train_input_data = tokenizer_encoder.texts_to_sequences(train_input_grams)
train_input_data = sequence.pad_sequences(train_input_data, maxlen = maxlen_seq, padding = 'post')

# Targets
train_target_data = tokenizer_decoder.texts_to_sequences(train_target_seqs)
train_target_data = sequence.pad_sequences(train_target_data, maxlen = maxlen_seq, padding = 'post')
train_target_data = to_categorical(train_target_data)

# Use the same tokenizer defined on train for tokenization of test
test_input_data = tokenizer_encoder.texts_to_sequences(test_input_grams)
test_input_data = sequence.pad_sequences(test_input_data, maxlen = maxlen_seq, padding = 'post')

# Computing the number of words and number of tags to be passed as parameters to the keras model
n_words = len(tokenizer_encoder.word_index) + 1
n_tags = len(tokenizer_decoder.word_index) + 1

input = Input(shape = (maxlen_seq,))




Create the CNN LSTM model

In [6]:

print(n_tags, n_words, maxlen_seq)
model = Sequential()
model.add(Embedding(input_dim = n_words, output_dim = 128, input_length = maxlen_seq))
model.add(Dropout(0.2))

model.add(Conv1D(filters=64,
                 kernel_size=5,
                 padding='same',
                 activation='relu',
                 strides=1))              

model.add(LSTM(units = 64, return_sequences = True, recurrent_dropout = 0.2))
model.add(Dense(n_tags, activation = "softmax"))
model.summary()




(9, 9623, 2048)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2048, 128)         1231744   
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048, 128)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 2048, 64)          41024     
_________________________________________________________________
lstm_1 (LSTM)                (None, 2048, 64)          33024     
_________________________________________________________________
dense_1 (Dense)              (None, 2048, 9)           585       
Total params: 1,306,377
Trainable params: 1,306,377
Non-trainable params: 0
_________________________________________________________________


Cross validation training

In [0]:

kf = KFold(n_splits=10)
kf.get_n_splits()

print(kf)  
print(train_input_data.shape, train_target_data.shape)
model.compile(optimizer = "rmsprop", loss = "categorical_crossentropy", metrics = ["accuracy", customAccuracy])

def Average(lst): 
    return sum(lst) / len(lst)

accuracies = []
counter = 0
for train_index, test_index in kf.split(train_target_data):
  Xtrain, Xtest = train_input_data[train_index], train_input_data[test_index]
  ytrain, ytest = train_target_data[train_index], train_target_data[test_index]
  model.fit(Xtrain, ytrain, batch_size = 128, epochs = 15, validation_data = (Xtest, ytest), verbose =  1)
  accuracy = model.evaluate(Xtest, ytest, batch_size=128)
  accuracies.append(accuracy[2])
  print(accuracies, Average(accuracies))

  # Defining the decoders so that we can
  revsere_decoder_index = {value:key for key,value in tokenizer_decoder.word_index.items()}
  revsere_encoder_index = {value:key for key,value in tokenizer_encoder.word_index.items()}
  y_test_pred = model.predict(test_input_data[:])
  print(len(test_input_data))
  counter += 1
  file = 'output' + str(counter) + ".csv"
  f = open(file,'w')
  f.write("id" + ',' + "expected")
  for i in range(len(test_df)):
    f.write('\n' + test_df['id'].iloc[i] +',' + str(onehot_to_seq(y_test_pred[i], revsere_decoder_index).upper()))
  f.close()
  !cp output* /content/drive/My\ Drive/

KFold(n_splits=10, random_state=None, shuffle=False)
((6471, 2048), (6471, 2048, 9))
Train on 5823 samples, validate on 648 samples
Epoch 1/15
5823/5823 [==============================] - 350s 60ms/step - loss: 0.4909 - acc: 0.8437 - customAccuracy: 0.2921 - val_loss: 0.2986 - val_acc: 0.8883 - val_customAccuracy: 0.3983
Epoch 2/15
5823/5823 [==============================] - 346s 59ms/step - loss: 0.3003 - acc: 0.8875 - customAccuracy: 0.4217 - val_loss: 0.2818 - val_acc: 0.8943 - val_customAccuracy: 0.4281
Epoch 3/15
5823/5823 [==============================] - 346s 59ms/step - loss: 0.2843 - acc: 0.8956 - customAccuracy: 0.4625 - val_loss: 0.2690 - val_acc: 0.9004 - val_customAccuracy: 0.4610
Epoch 4/15
5823/5823 [==============================] - 346s 59ms/step - loss: 0.2704 - acc: 0.9011 - customAccuracy: 0.4911 - val_loss: 0.2595 - val_acc: 0.9041 - val_customAccuracy: 0.4810
Epoch 5/15
5823/5823 [==============================] - 347s 60ms/step - loss: 0.2630 - acc: 0.9046 - cu

Save the output

In [0]:
f = open('output.csv','w')
f.write("id" + ',' + "expected")
for i in range(len(test_df)):
  f.write('\n' + test_df['id'].iloc[i] +',' + str(onehot_to_seq(y_test_pred[i], revsere_decoder_index).upper()))
f.close()

    #print_results(test_input_seqs[i], y_test_pred[i], revsere_decoder_index)

In [0]:
!ls
!head output.csv
!ls -ltr output.csv
!date
!cp output.csv /content/drive/My\ Drive/